# Setup

In [7]:
from dslabs_functions import get_variable_types
from seaborn import heatmap
from dslabs_functions import HEIGHT, plot_multi_scatters_chart
from matplotlib.pyplot import figure, subplots, savefig, show, gcf
from dslabs_functions import plot_bar_chart
from dslabs_functions import set_chart_labels
from dslabs_functions import define_grid, HEIGHT
from matplotlib.figure import Figure
from numpy import ndarray
from dslabs_functions import *
from pandas import read_csv, DataFrame
from numpy import log
from pandas import Series
from scipy.stats import norm, expon, lognorm
from matplotlib.axes import Axes
from dslabs_functions import plot_multiline_chart
import pandas as pd

In [2]:
results = "csv-results/credit-score-after-preparation.csv"
file_tag = "credit_score"
data: DataFrame = read_csv(results, index_col="ID")

**IMPORTANT REMINDER:** Remember that balancing cannot be applied to the test dataset, and that the evaluation of the models has to be done over an independent dataset (test set).

In [8]:
from sklearn.model_selection import train_test_split

data_to_split = read_csv('csv-results/credit-score-before-balancing')

# Split train/test

target = 'Credit_Score'
y = data_to_split[target]
X = data_to_split.drop(columns = target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

# Naive Bayes

## Approaches study

## Best results

# KNN

## Parameters study

## Best results

## Overfitting

# Decision Trees


## Parameters study

## Best results

## Best tree

## Variables relevancy

## Overfitting